In [2]:
%pip install --upgrade pip
%pip install boto3 prettytable construct

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import boto3
import subprocess
import json
import uuid
import construct as c
import websocket
import re
import os
import time
from prettytable import PrettyTable
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed

In [8]:
def set_aws_credentials(profile, region_name='us-east-1'):
    result = subprocess.run(
        f"aws-vault exec {profile} --json", shell=True, capture_output=True)
    credentials = json.loads(result.stdout)

    # Create a session with the retrieved credentials
    session = boto3.session.Session(
        aws_access_key_id=credentials['AccessKeyId'],
        aws_secret_access_key=credentials['SecretAccessKey'],
        aws_session_token=credentials['SessionToken'],
        region_name=region_name
    )

    return session


def list_tasks(ecs_client, cluster_name, service_name):
    # List all running tasks in a specified service
    response = ecs_client.list_tasks(
        cluster=cluster_name,
        serviceName=service_name,
        desiredStatus='RUNNING'
    )
    return response['taskArns']


def session_reader(session: dict) -> str:
    AgentMessageHeader = c.Struct(
        "HeaderLength" / c.Int32ub,
        "MessageType" / c.PaddedString(32, "ascii"),
    )

    AgentMessagePayload = c.Struct(
        "PayloadLength" / c.Int32ub,
        "Payload" / c.PaddedString(c.this.PayloadLength, "ascii"),
    )

    connection = websocket.create_connection(session["streamUrl"])
    try:
        init_payload = {
            "MessageSchemaVersion": "1.0",
            "RequestId": str(uuid.uuid4()),
            "TokenValue": session["tokenValue"],
        }
        connection.send(json.dumps(init_payload))
        while True:
            resp = connection.recv()
            message = AgentMessageHeader.parse(resp)
            if "channel_closed" in message.MessageType:
                raise Exception(
                    "Channel closed before command output was received")
            if "output_stream_data" in message.MessageType:
                break
    finally:
        connection.close()
    payload_message = AgentMessagePayload.parse(resp[message.HeaderLength:])
    return payload_message.Payload


def execute_curl_on_task(ecs_client, cluster_name, task_arn, container_name):
    # Command to execute
    command = "curl http://localhost:2000/nginx_status"

    # Start ECS Exec session
    exec_resp = ecs_client.execute_command(
        cluster=cluster_name,
        task=task_arn,
        container=container_name,
        command=command,
        interactive=True
    )

    response = session_reader(exec_resp["session"])
    return response


def parse_curl_output(output):
    # Splitting the output into lines
    lines = output.strip().split('\n')

    # Extracting the values
    active = int(re.search(r'Active connections: (\d+)', lines[0]).group(1))
    accepts, handled, requests = map(int, re.findall(r'(\d+)', lines[2]))
    reading, writing, waiting = map(int, re.findall(
        r'Reading: (\d+) Writing: (\d+) Waiting: (\d+)', lines[3])[0])

    return {
        'active': active,
        'accepts': accepts,
        'handled': handled,
        'requests': requests,
        'reading': reading,
        'writing': writing,
        'waiting': waiting
    }


def get_nginx_status(profile, region_name, cluster_name, service_name, container_name='nginx'):
    aws_session = set_aws_credentials(profile, region_name=region_name)

    # Create ECS client using the session
    ecs_client = aws_session.client('ecs')

    # List all running tasks in a specified service
    task_arns = list_tasks(ecs_client, cluster_name, service_name)

    print(
        f"Found {len(task_arns)} tasks running in {cluster_name}/{service_name}")

    # Initialize a defaultdict for summing up the values
    aggregated_stats = defaultdict(int)

    '''
    for task_arn in task_arns:
        response = execute_curl_on_task(ecs_client, cluster_name, task_arn, container_name)
        stats = parse_curl_output(response)

        print(f"Task {task_arn} - {stats}")
        # Iterate over each dictionary and sum up the values
        for key, value in stats.items():
            aggregated_stats[key] += value
    '''

    # Use ThreadPoolExecutor to execute tasks in parallel
    with ThreadPoolExecutor(max_workers=8) as executor:
        # Submit all tasks to the executor
        future_to_task = {executor.submit(
            execute_curl_on_task, ecs_client, cluster_name, task_arn, container_name): task_arn for task_arn in task_arns}

        for future in as_completed(future_to_task):
            task_arn = future_to_task[future]
            try:
                response = future.result()
                stats = parse_curl_output(response)
                print(f"Task {task_arn} - {stats}")
                # Sum up the values
                for key, value in stats.items():
                    aggregated_stats[key] += value
            except Exception as e:
                print(f"Task {task_arn} failed: {e}")

    return aggregated_stats


# Specify your cluster and service name
cluster_name = 'EcsCluster1-main'
service_name = 'projects-application-fargate-main'

environments = [
    ('acl-staging', 'us-west-2'),  # staging
    ('acl-staging', 'us-east-1'),  # preprod?
    # ('acl-production', 'us-east-1'),
    # ('acl-production', 'eu-central-1'),
    # ('acl-production', 'af-south-1'),
    # ('acl-production', 'ca-central-1'),
    # ('acl-production', 'ap-northeast-1'),
    # ('acl-production', 'ap-southeast-1'),
]

table = PrettyTable()
table.field_names = ["Profile", "Region", "active", "handled",
                     "accepts", "requests", "reading", "writing", "waiting"]

for profile, region in environments:
    aggregated_stats = get_nginx_status(
        profile, region, cluster_name, service_name)
    table.add_row([profile, region,
                   aggregated_stats['active'],
                   aggregated_stats['handled'],
                   aggregated_stats['accepts'],
                   aggregated_stats['requests'],
                   aggregated_stats['reading'],
                   aggregated_stats['writing'],
                   aggregated_stats['waiting']])

print(table)

Found 2 tasks running in EcsCluster1-main/projects-application-fargate-main
Task arn:aws:ecs:us-west-2:649973207834:task/EcsCluster1-main/b0a8fc34a5fd416ca6fe06896bcadccf - {'active': 3, 'accepts': 418, 'handled': 418, 'requests': 556, 'reading': 0, 'writing': 1, 'waiting': 2}
Task arn:aws:ecs:us-west-2:649973207834:task/EcsCluster1-main/f34bb88a1c00473c97ce1f9723afa23f - {'active': 1, 'accepts': 399, 'handled': 399, 'requests': 551, 'reading': 0, 'writing': 1, 'waiting': 0}
Found 3 tasks running in EcsCluster1-main/projects-application-fargate-main
Task arn:aws:ecs:us-east-1:649973207834:task/EcsCluster1-main/423ab22edf2347d5bf2596769d766ca6 - {'active': 1, 'accepts': 14698, 'handled': 14698, 'requests': 113490, 'reading': 0, 'writing': 1, 'waiting': 0}
Task arn:aws:ecs:us-east-1:649973207834:task/EcsCluster1-main/d74022fab77f4fee9b1a63b83adbf8fc - {'active': 2, 'accepts': 14763, 'handled': 14763, 'requests': 113494, 'reading': 0, 'writing': 1, 'waiting': 1}
Task arn:aws:ecs:us-east-1

attach to task:

`aws ecs execute-command --cluster EcsCluster1-main --task 0bc3316b21414e6490f4a9707e11ec52 --command "/bin/bash" --interactive --container nginx`

`Active connections` - The current number of active client connections including Waiting connections.

`accepts` - The total number of accepted client connections.

`handled` - The total number of handled connections. Generally, the parameter value is the same as accepts unless some resource limits have been reached (for example, the worker_connections limit).

`requests` - The total number of client requests.

`Reading` - The current number of connections where nginx is reading the request header.

`Writing` - The current number of connections where nginx is writing the response back to the client.

`Waiting` - The current number of idle client connections waiting for a request.